# Prompt Tuning 实战

## Step1 导入相关包

In [3]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

## Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--Langboat--bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='D:/pretrained_model/models--Langboat--bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [5]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [6]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [7]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [8]:
model = AutoModelForCausalLM.from_pretrained("D:/pretrained_model/models--Langboat--bloom-1b4-zh", low_cpu_mem_usage=True)

## Prompt tuning

### PEFT Step1 配置文件

In [11]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit
'''
task_type: 设置为 TaskType.CAUSAL_LM，表明我们要执行的任务是因果语言建模（Causal Language Modeling），即给定一段文本序列预测下一个词。
prompt_tuning_init: 设置为 PromptTuningInit.TEXT，表示我们想要用文本的方式初始化提示。
prompt_tuning_init_text: 提供了一段中文文本 "下面是一段人机对话。" 作为提示的初始文本。
num_virtual_tokens: 这个参数指定了要插入到输入中的虚拟标记数量。这里是通过将 prompt_tuning_init_text 文本传递给分词器（tokenizer）并计算其长度来确定的。
tokenizer_name_or_path: 指向了本地存储的预训练分词器路径，这个分词器应该与你使用的预训练模型相匹配。
'''

# Soft Prompt
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10)
config

# Hard Prompt
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
                            prompt_tuning_init=PromptTuningInit.TEXT,
                            prompt_tuning_init_text="下面是一段人机对话。",
                            num_virtual_tokens=len(tokenizer("下面是一段人机对话。")['input_ids']),
                            tokenizer_name_or_path="D:/pretrained_model/models--Langboat--bloom-1b4-zh")
print(config)

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=7, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, prompt_tuning_init=<PromptTuningInit.TEXT: 'TEXT'>, prompt_tuning_init_text='下面是一段人机对话。', tokenizer_name_or_path='D:/pretrained_model/models--Langboat--bloom-1b4-zh', tokenizer_kwargs=None)


### PEFT Step2 创建模型

In [12]:
model = get_peft_model(model, config)

In [13]:
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [14]:
model.print_trainable_parameters()

trainable params: 14,336 || all params: 1,303,126,016 || trainable%: 0.0011


## Step5 配置训练参数

In [15]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=20,
)

## Step6 创建训练器

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Step7 模型训练

In [17]:
trainer.train()

  0%|          | 0/3357 [00:00<?, ?it/s]

{'loss': 2.9337, 'grad_norm': 1.9710654020309448, 'learning_rate': 4.985105749180816e-05, 'epoch': 0.0}
{'loss': 3.1078, 'grad_norm': 1.54780912399292, 'learning_rate': 4.9702114983616324e-05, 'epoch': 0.01}


C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:/pretrained_model/models--Langboat--bloom-1b4-zh - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 2.928, 'grad_norm': 0.9953110218048096, 'learning_rate': 4.9553172475424484e-05, 'epoch': 0.01}
{'loss': 2.9116, 'grad_norm': 1.7055747509002686, 'learning_rate': 4.940422996723265e-05, 'epoch': 0.01}


C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:/pretrained_model/models--Langboat--bloom-1b4-zh - will assume that the vocabulary was not modified.
  warnings.warn(


...

{'loss': 2.563, 'grad_norm': 3.888773202896118, 'learning_rate': 4.419124218051832e-05, 'epoch': 0.12}
{'loss': 2.6883, 'grad_norm': 9.817695617675781, 'learning_rate': 4.404229967232648e-05, 'epoch': 0.12}


C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:/pretrained_model/models--Langboat--bloom-1b4-zh - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 2.5483, 'grad_norm': 2.6598668098449707, 'learning_rate': 4.389335716413465e-05, 'epoch': 0.12}
{'loss': 2.651, 'grad_norm': 3.1092569828033447, 'learning_rate': 4.374441465594281e-05, 'epoch': 0.13}


C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:/pretrained_model/models--Langboat--bloom-1b4-zh - will assume that the vocabulary was not modified.
  warnings.warn(


KeyboardInterrupt: 

## 加载训练好的PEFT模型

In [2]:
from peft import PeftModel

In [4]:
# 在一个jupyter文件中，如果前面已经加载了模型，并对模型做了一定修改，则需要重新加载原始模型
model = AutoModelForCausalLM.from_pretrained("D:/pretrained_model/models--Langboat--bloom-1b4-zh")
peft_model = PeftModel.from_pretrained(model=model, model_id="./chatbot/checkpoint-200/")

## Step8 模型推理

In [7]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))

Human: 考试有哪些技巧？

Assistant: 计算机考试是考查一个人的智力与实践操作能力的考试，它是在计算机上按照一定规律和规范来考查一个人的知识与技能，目的是判断一个人的知识与技能的掌握程度。计算机考试的题是围绕考生的实际情况进行设计的，题目都经过专业教师和编题人员精心设计、编排。题型包括计算机应用及基础知识、计算机编程、计算机组成、操作系统、计算机网络基础知识、数据库、图形图像处理基础、数据结构等。考试题由计算机程序编写，包括两个半导体组件，一个用作


In [9]:
import torch
import gc

# 假设 peft_model 已经在 GPU 上运行过
# 释放 peft_model 的显存

# 将模型移动到 CPU 或者直接删除它以释放显存
if peft_model is not None:
    # 如果您想稍后重新使用该模型，请将其移到CPU上
    peft_model = peft_model.to('cpu')
    
    # 或者，如果您不再需要这个模型，您可以直接删除它
    del peft_model


# 清空CUDA缓存
torch.cuda.empty_cache()

# 强制垃圾回收
gc.collect()

# 再次清空CUDA缓存以确保尽可能多的显存被释放
torch.cuda.empty_cache()